# Module 2 Assessment

Welcome to your Mod 2 Assessment. You will be tested for your understanding of concepts and ability to solve problems that have been covered in class and in the curriculum.

Use any libraries you want to solve the problems in the assessment.

You will have up to two hours to complete this assessment.

The sections of the assessment are:

- Accessing Data Through APIs
- Object Oriented Programming
- SQL and Relational Databases
- HTML, CSS and Web Scraping
- Other Database Structures (MongoDB)

In this assessment you will be exploring two datasets: Pokemon and Quotes.

In [3]:
# import the necessary libraries

import requests
import json
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
# import pymongo

## Part 1: Accessing Data Through APIs

In this section we'll be using PokeAPI to get data on Pokemon. Let's first define functions to get information from the API. Provided below is a URL that will get you started with the first 151 Pokemon! Run the cell below to see what we get.

In [74]:
url = 'https://pokeapi.co/api/v2/pokemon/?limit=151'
results = requests.get(url).json()['results']
results

[{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
 {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
 {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
 {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
 {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
 {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
 {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
 {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
 {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
 {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
 {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
 {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
 {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2/pokemon/13/'},
 {'name': 'kakuna', 'url': 'https://pokeapi.co/api/v2/pokemon/14/'},
 {'name': '

[Read the documentation here](https://pokeapi.co/) for information on navigating this API and use the API to obtain data to answer the following questions.

### Accessing Data

1. For any **one** Pokemon, retrieve the following information in a dictionary format with the following keys:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities

For `Types` and `Abilities`, you might want to write helper functions to have each attribute just be a list of types and a list of abilities. Your output should look like this:

```
{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}

```
    


In [75]:
# you may define helper functions for types and abilities here
def extract(category, kwarg_str):
    lst = []
    for x in category:
        lst.append(x[kwarg_str]['name'])
    return lst

In [76]:
def get_pokedata(url):
    
    """
    url is a string of the URL that will get you the relevant info from the API
    
    this function should return the dictionary for ONE pokemon
    
    """    
    one_url = url
    one_dict = requests.get(one_url).json()
    final_dict = {}
    final_dict['id'] = one_dict['id'] # hmm
    final_dict['name'] = one_dict['name']
    final_dict['base_experience'] = one_dict['base_experience']
    final_dict['weight'] = one_dict['weight']
    final_dict['height'] = one_dict['height']
    final_dict['types'] = extract(one_dict['types'],'type')
    final_dict['abilities'] = extract(one_dict['abilities'],'ability')
    return final_dict
    
# print(get_pokedata('https://pokeapi.co/api/v2/pokemon/149/'))

### Pagination

2. Get the same information for the first **151** Pokemon as a list of dictionaries ordered by Pokemon ID. Print the first and last elements of the list. (Hint: Use pagination) Your output should save the list to a variable and look like this:

```
[{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, 
{'id': 2, 
'name': 'ivysaur', 
'base_experience': 142, 
'weight': 130, 
'height': 10, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, ... ]

```



In [77]:
import copy
""" 

Assign to "pokedata" the list of 151 dictionaries.
You may use your function from the previous question.

"""
def get_all_pokedata(url):
    max_idx = 151
    final_list = []
    for i in range(1, max_idx+1):
        one_pokedict = get_pokedata(url+str(i)+'/')
        final_list.append(one_pokedict.copy())    
    return final_list
    
pokedata = get_all_pokedata('https://pokeapi.co/api/v2/pokemon/')
# pokedata

In [78]:
# printing first and last elements

print(pokedata[0], pokedata[-1])

{'id': 1, 'name': 'bulbasaur', 'base_experience': 64, 'weight': 69, 'height': 7, 'types': ['poison', 'grass'], 'abilities': ['chlorophyll', 'overgrow']} {'id': 151, 'name': 'mew', 'base_experience': 270, 'weight': 40, 'height': 4, 'types': ['psychic'], 'abilities': ['synchronize']}


## Part 2: Object Oriented Programming

We're going to use the data gathered in the previous section on APIs for this section on Object Oriented Programming to instantiate Pokemon objects and write instance methods.

### Creating a Class

1. Create a class called `Pokemon` with an `__init__` method to instantiate the following attributes:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities
    



In [ ]:
# if you were unable to get the data from the API in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open('data/pokemon.json') as f:  
#     pokelist = json.load(f)

In [79]:
"""
Create your class below with the correct syntax, including an __init__ method.
    final_dict['id'] = one_dict['id'] # hmm
    final_dict['name'] = one_dict['name']
    final_dict['base_experience'] = one_dict['base_experience']
    final_dict['weight'] = one_dict['weight']
    final_dict['height'] = one_dict['height']
    final_dict['types'] = extract(one_dict['types'],'type')
    final_dict['abilities'] 

"""
class Pokemon:
    def __init__(self, idee, name, base_experience, weight, height, types, abilities):
        self.id = idee
        self.name = name
        self.base_experience = base_experience
        self.weight = weight
        self.height = height
        self.types = types
        self.abilities = abilities
    def bmi(self):
        return round(10*self.weight/(self.height**2),2)

    
### Instantiating Objects

2. Using the data you obtained from the API, instantiate the first, fourth and seventh Pokemon. Assign them to the variables `bulbasaur`, `charmander` and `squirtle`.

In [80]:
# Your code here
bulbasaur, charmander, squirtle = {}, {}, {}
bulbasaur = Pokemon(pokedata[0]['id'],pokedata[0]['name'],pokedata[0]['base_experience'],
                    pokedata[0]['weight'],pokedata[0]['height'],pokedata[0]['types'],pokedata[0]['abilities'])
charmander = Pokemon(pokedata[3]['id'],pokedata[3]['name'],pokedata[3]['base_experience'],
                    pokedata[3]['weight'],pokedata[3]['height'],pokedata[3]['types'],pokedata[3]['abilities'])
squirtle = Pokemon(pokedata[6]['id'],pokedata[6]['name'],pokedata[6]['base_experience'],
                    pokedata[6]['weight'],pokedata[6]['height'],pokedata[6]['types'],pokedata[6]['abilities'])

# print(bulbasaur,'\n',charmander,'\n',squirtle)

In [81]:
# run this cell to test and check your code
# you may need to edit the attribute variable names if you named them differently!

def print_pokeinfo(pokemon_object):
    o = pokemon_object
    print('ID: ' + str(o.id) + '\n' +
          'Name: ' + o.name.title() + '\n' +
          'Base experience: ' + str(o.base_experience) + '\n' +
          'Weight: ' + str(o.weight) + '\n' +
          'Height: ' + str(o.height) + '\n' +
          'Types: ' + str(o.types) + '\n' +
          'Abilities: ' + str(o.abilities) + '\n'
         )
    
print_pokeinfo(bulbasaur)
print_pokeinfo(charmander)
print_pokeinfo(squirtle)

ID: 1
Name: Bulbasaur
Base experience: 64
Weight: 69
Height: 7
Types: ['poison', 'grass']
Abilities: ['chlorophyll', 'overgrow']

ID: 4
Name: Charmander
Base experience: 62
Weight: 85
Height: 6
Types: ['fire']
Abilities: ['solar-power', 'blaze']

ID: 7
Name: Squirtle
Base experience: 63
Weight: 90
Height: 5
Types: ['water']
Abilities: ['rain-dish', 'torrent']



### Instance Methods

3. Write an instance method within the class `Pokemon` to find the BMI of a Pokemon. BMI is defined by $\frac{weight}{height^{2}}$ with weight in **kilograms** and height in **meters**. The height and weight data of Pokemon from the API is in **decimeters** and **hectograms** respectively.


    1 decimeter = 0.1 meters
    1 hectogram = 0.1 kilograms

In [82]:
# run this cell to test and check your code
# you will probably have to rerun the code to instantiate your objects

print(bulbasaur.bmi()) # 14.08
print(charmander.bmi()) # 23.61
print(squirtle.bmi()) # 36

14.08
23.61
36.0


## Part 3: SQL and Relational Databases

For this section, we've put the Pokemon data into SQL tables. You won't need to use your list of dictionaries or the JSON file for this section. The schema of `pokemon.db` is as follows:

<img src="data/pokemondb.png" alt="db schema" style="width:500px;"/>

Assign your SQL queries as strings to the variables `q1`, `q2`, etc. and run the cells at the end of this section to print your results as Pandas DataFrames.

- q1: query all columns from `Pokemon` the Pokemon that have base_experience above 200  

  
- q2: query the id, name, type1 and type2 of Pokemon that have **water** types as either their first or second type


- q3: query the average weight of Pokemon by their first type in descending order


- q4: query the Pokemon name, Pokemon type2, and what **type2** has "2xdamage" to


- q5: query the top 5 most common type1s, the minimum height, maximum height, minimum weight and maximum weight of pokemon with those type1s, and what associated type they do "0.5xdamage" to


**Important note on syntax**: use `double quotes ""` when quoting strings **within** your query and wrap the entire query in `single quotes ''` For the column titles that begin with numbers, you need to wrap the column names in double quotes.

In [83]:
cnx = sqlite3.connect('data/pokemon.db')

In [84]:
q1 = 'SELECT * FROM Pokemon WHERE base_experience > 200'
pd.read_sql(q1, cnx)

,id,name,base_experience,weight,height,type1,type2
0,3,venusaur,236,1000,20,grass,poison
1,6,charizard,240,905,17,fire,flying
2,9,blastoise,239,855,16,water,None
3,18,pidgeot,216,395,15,normal,flying
4,26,raichu,218,300,8,electric,None
5,31,nidoqueen,227,600,13,poison,ground
6,34,nidoking,227,620,14,poison,ground
7,36,clefable,217,400,13,fairy,None
8,45,vileplume,221,186,12,grass,poison
9,62,poliwrath,230,540,13,water,fighting


In [85]:
"""query the id, name, type1 and type2 of Pokemon that have water types as either their first or second type"""
q2 = 'SELECT id, name, type1, type2 FROM Pokemon WHERE type1 = "water" OR type2 = "water";'
pd.read_sql(q2, cnx)

,id,name,type1,type2
0,7,squirtle,water,None
1,8,wartortle,water,None
2,9,blastoise,water,None
3,54,psyduck,water,None
4,55,golduck,water,None
5,60,poliwag,water,None
6,61,poliwhirl,water,None
7,62,poliwrath,water,fighting
8,72,tentacool,water,poison
9,73,tentacruel,water,poison


In [86]:
"""query the average weight of Pokemon by their first type in descending order"""
q3 = 'SELECT type1, ROUND(AVG(weight), 2) AS "Avg Wt" FROM Pokemon GROUP BY type1 ORDER BY type1 DESC'
pd.read_sql(q3, cnx)

,type1,Avg Wt
0,water,579.68
1,rock,876.11
2,psychic,515.63
3,poison,273.14
4,normal,500.86
5,ice,480.00
6,ground,452.63
7,grass,279.92
8,ghost,135.67
9,fire,480.25


In [87]:
"""query the Pokemon name, Pokemon type2, and what type2 has "2xdamage" to"""
q4 = 'SELECT Pokemon.name, type2, Types."2xdamage" FROM Pokemon JOIN Types ON Pokemon.type2 = Types.name'
pd.read_sql(q4, cnx)

,name,type2,2xdamage
0,bulbasaur,poison,grass
1,ivysaur,poison,grass
2,venusaur,poison,grass
3,charizard,flying,fighting
4,butterfree,flying,fighting
5,weedle,poison,grass
6,kakuna,poison,grass
7,beedrill,poison,grass
8,pidgey,flying,fighting
9,pidgeotto,flying,fighting


In [105]:
"""query the top 5 most common type1s, the minimum height, maximum height, minimum weight and maximum weight of 
pokemon with those type1s, and what associated type they do "0.5xdamage" to """
q5 = '''SELECT type1, COUNT(type1), MIN(height), MAX(height), MIN(weight), MAX(weight), Types."0.5xdamage"
        FROM Pokemon JOIN Types ON Pokemon.type2 = Types.name GROUP BY type1 ORDER BY COUNT(type1) DESC LIMIT 5;'''
pd.read_sql(q5, cnx)

,type1,COUNT(type1),MIN(height),MAX(height),MIN(weight),MAX(weight),0.5xdamage
0,grass,11,4,20,25,1200,fighting
1,normal,10,3,18,18,852,fighting
2,water,10,9,65,360,2350,fighting
3,bug,9,3,15,32,560,fighting
4,rock,9,4,88,75,3000,poison


## Section 4: Web Scraping

### Accessing Data Using BeautifulSoup

Use BeautifulSoup to get quotes, authors, and tags from [Quotes to Read](http://quotes.toscrape.com/).

Before answering these questions, go to the site and inspect the page. Make sure to look at what links there are and how the site is structured.

1. Get the first author and the path for the author's page as a tuple from the [homepage](http://quotes.toscrape.com/).

In [152]:
# Make a get request to retrieve the page
html_page = requests.get('http://quotes.toscrape.com/') 
# Pass the page contents to beautiful soup for parsing
soup = BeautifulSoup(html_page.content, 'html.parser')
auth, auth_url  = '', ''
ans = tuple()
auth = soup.find('small').text
for s in soup.findAll('a', href = True):
    if s.text == "(about)":
        auth_url = 'http://quotes.toscrape.com'+s['href']
        break

ans = (auth, auth_url)
print(ans)

('Albert Einstein', 'http://quotes.toscrape.com/author/Albert-Einstein')


In [91]:
for a in soup.find_all('a', href=True):
    if "author" in a['href']:
        print(a['href'])
# for s in soup.findAll('span'):
#     if s.a:
#         print(s.a)
# quote = soup.find('div', class_="quote").get_text().split('\n')[1].replace('“','').replace('”','')
# tup = (auth, quote)
#print(tup)


/author/Albert-Einstein
/author/J-K-Rowling
/author/Albert-Einstein
/author/Jane-Austen
/author/Marilyn-Monroe
/author/Albert-Einstein
/author/Andre-Gide
/author/Thomas-A-Edison
/author/Eleanor-Roosevelt
/author/Steve-Martin


2. Write a function to get **all** the authors and href links for the authors from the [homepage](http://quotes.toscrape.com/)


In [165]:
def authors(url):
    '''
    input: url
    
    return: a dictionary of of authors and their urls
            {'author_1':'url_of_author_1', 'author_2':'url_of_author_2' ...}
    '''
    final = {}
    html_page = requests.get(url) 
    soup = BeautifulSoup(html_page.content, 'html.parser')
    for i in soup.findAll('small', class_="author"):
        for s in soup.findAll('a', href = True):
            if s.text == "(about)":
                final[i.text] = 'http://quotes.toscrape.com'+s['href']
                continue
    return final

print(authors("http://quotes.toscrape.com/"))

{'Albert Einstein': 'http://quotes.toscrape.com/author/Steve-Martin', 'J.K. Rowling': 'http://quotes.toscrape.com/author/Steve-Martin', 'Jane Austen': 'http://quotes.toscrape.com/author/Steve-Martin', 'Marilyn Monroe': 'http://quotes.toscrape.com/author/Steve-Martin', 'André Gide': 'http://quotes.toscrape.com/author/Steve-Martin', 'Thomas A. Edison': 'http://quotes.toscrape.com/author/Steve-Martin', 'Eleanor Roosevelt': 'http://quotes.toscrape.com/author/Steve-Martin', 'Steve Martin': 'http://quotes.toscrape.com/author/Steve-Martin'}


In [166]:
# run this cell to test the function
print(authors('http://quotes.toscrape.com/'))
print('\n')
print(authors('http://quotes.toscrape.com/page/3'))

{'Albert Einstein': 'http://quotes.toscrape.com/author/Steve-Martin', 'J.K. Rowling': 'http://quotes.toscrape.com/author/Steve-Martin', 'Jane Austen': 'http://quotes.toscrape.com/author/Steve-Martin', 'Marilyn Monroe': 'http://quotes.toscrape.com/author/Steve-Martin', 'André Gide': 'http://quotes.toscrape.com/author/Steve-Martin', 'Thomas A. Edison': 'http://quotes.toscrape.com/author/Steve-Martin', 'Eleanor Roosevelt': 'http://quotes.toscrape.com/author/Steve-Martin', 'Steve Martin': 'http://quotes.toscrape.com/author/Steve-Martin'}


{'Pablo Neruda': 'http://quotes.toscrape.com/author/Bob-Marley', 'Ralph Waldo Emerson': 'http://quotes.toscrape.com/author/Bob-Marley', 'Mother Teresa': 'http://quotes.toscrape.com/author/Bob-Marley', 'Garrison Keillor': 'http://quotes.toscrape.com/author/Bob-Marley', 'Jim Henson': 'http://quotes.toscrape.com/author/Bob-Marley', 'Dr. Seuss': 'http://quotes.toscrape.com/author/Bob-Marley', 'Albert Einstein': 'http://quotes.toscrape.com/author/Bob-Marley',

### Pagination

3. Get the first author on each of the first 5 pages of quotes. You can get to the next page with the next button at the bottom of the homepage.


In [171]:
lst = []
for i in range(1,6):
    for a in authors('http://quotes.toscrape.com/page/'+str(i)):
        lst.append(a)
        break
lst

['Albert Einstein',
 'Marilyn Monroe',
 'Pablo Neruda',
 'Dr. Seuss',
 'George R.R. Martin']

4. Write a function to get all of the quotes from a page.

In [196]:
def get_some_quotes(url, num_pgs = 1):
    '''
    input: url, number of pages to scrap (just scrape the home page if no argument is passed in)
    
    return: a list of dictionaries of quotes with their attributes
            [{'quote':'quote_1_text', 'author':'url_of_author_1'}, 
            {'quote':'quote_2_text', 'author':'url_of_author_2', 'quote_tags':[list_of_quote_2_tags]}, ...]
    '''
    auts, quotes = [], []
    final = {}
    lst = []
    for j in range(1,num_pgs+1):
        if j == 1:
            html_page = requests.get(url)
        else:
            html_page = requests.get(url+"page/"+str(j))
        soup = BeautifulSoup(html_page.content, 'html.parser')
        for x in soup.findAll('small', class_="author", itemprop="author"):
            auts.append(x.text)
        for t in soup.findAll('span', class_="text", itemprop="text"):
            quotes.append(t.text)
        for k in range(len(auts)):
            final[auts[k]] = quotes[k]
    return final
#     if len(soup.findAll('small', class_="author")) == len(soup.findAll('span', class_="text", itemprop="text")):
#         for i in range(len(soup.findAll('small', class_="author"))):
#             for s in soup.findAll('small', class_="author"):
#                 a_l.append(s.text)
#             for t in soup.findAll('span', class_="text", itemprop="text"):
#                 q_l.append(t.text)
#         final = list(set(zip(a_l, q_l)))
#     else:
#         raise KeyError
#     return final

print(get_some_quotes('http://quotes.toscrape.com/', 5))        

{'Albert Einstein': '“Life is like riding a bicycle. To keep your balance, you must keep moving.”', 'J.K. Rowling': '“It matters not what someone is born, but what they grow to be.”', 'Jane Austen': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”', 'Marilyn Monroe': "“A wise girl kisses but doesn't love, listens but doesn't believe, and leaves before she is left.”", 'André Gide': '“It is better to be hated for what you are than to be loved for what you are not.”', 'Thomas A. Edison': "“I have not failed. I've just found 10,000 ways that won't work.”", 'Eleanor Roosevelt': "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”", 'Steve Martin': '“A day without sunshine is like, you know, night.”', 'Bob Marley': '“The truth is, everyone is going to hurt you. You just got to find the ones worth suffering for.”', 'Dr. Seuss': "“The more that you read, the more things you will know. The more that you l

In [197]:
# set the function to a variable to use later
quotes_for_mongo = get_some_quotes('http://quotes.toscrape.com/' )
quotes_for_mongo

{'Albert Einstein': '“Try not to become a man of success. Rather become a man of value.”',
 'J.K. Rowling': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 'Jane Austen': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 'Marilyn Monroe': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 'André Gide': '“It is better to be hated for what you are than to be loved for what you are not.”',
 'Thomas A. Edison': "“I have not failed. I've just found 10,000 ways that won't work.”",
 'Eleanor Roosevelt': "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 'Steve Martin': '“A day without sunshine is like, you know, night.”'}

## Part 5: MongoDB

To do this section, open a connection to a mongo database in the terminal, using `mongod`. You will then **create**, **update**, and **read** from a mongo database.

Create and connect to a mongo database.

In [ ]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
mydb = myclient['quote_database']

In [ ]:
mycollection = mydb['quote_collection']

1. Add the quotes you obtained from the `get_some_quotes` function for the [homepage](http://quotes.toscrape.com/) to the mongo database. (You can also use the JSON file `quotes.json` to insert data into the database) To verify that you've successfully inserted the data, query it to obtain the resulting _ids back from the `results` variable. 

In [ ]:
# if you were unable to get the data from webscraping in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open(r"data/quotes.json", "r") as r:
#     data = json.load(r)

In [ ]:
# use the results variable to confirm the data was inserted
results = None

2. Query the database for all the quotes written by `'Albert Einstein'`.

In [ ]:
q1 = None

3. Update Steve Martin's quote with the tags for the quote stored in the variable `steve_martin_tags`.

In [ ]:
steve_martin_tags = {'quote_tags': ['change', 'deep-thoughts', 'thinking', 'world']}
update_steve = None
first_quote_tags = None


4. Query the database to confirm that  `'Steve Martin'` is updated with `steve_martin_tags`.

In [ ]:
q2 = None